In [0]:

dbutils.fs.mount(
    source='wasbs://srini-adlsgen2adventureworks@adlsgen2adventureworks.blob.core.windows.net/',
    mount_point='/mnt/bronze-srini',
    extra_configs={
'fs.azure.account.key.adlsgen2adventureworks.blob.core.windows.net': 'KFrirnuNDTcvixYNQIlGHiDoMLPrOeh5VvED866hfXh6Vn6OEFJXLGaSpHDTWzTp2ir503LwgUSu+ASt7YmK+w=='
    }
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4409814233663320>:1
----> 1 dbutils.fs.mount(
      2     source='wasbs://srini-adlsgen2adventureworks@adlsgen2adventureworks.blob.core.windows.net/',
      3     mount_point='/mnt/bronze-srini',
      4     extra_configs={
      5 'fs.azure.account.key.adlsgen2adventureworks.blob.core.windows.net': 'KFrirnuNDTcvixYNQIlGHiDoMLPrOeh5VvED866hfXh6Vn6OEFJXLGaSpHDTWzTp2ir503LwgUSu+ASt7YmK+w=='
      6     }
      7 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o398.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/bronze

In [0]:

PD_PATH = "/mnt/bronze-srini/BRONZE"

def list_files_recursively(directory):
    file_paths = []
   
    files = dbutils.fs.ls(directory)
    for file in files:
        if file.isDir():
            # If the item is a directory, recursively call the function
            file_paths.extend(list_files_recursively(file.path))
        else:
            # If the item is a file, append its path to the list
            file_paths.append(file.path)
    return file_paths
 
# Get list of all files in the parent directory and its subdirectories
file_paths = list_files_recursively(PD_PATH )
print(file_paths)

['dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Department/2024-02-21T10:59:31.7566414/HumanResources_Department.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Employee/2024-02-21T10:59:51.2337011/HumanResources_Employee.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/EmployeeDepartmentHistory/2024-02-21T11:00:04.9490793/HumanResources_EmployeeDepartmentHistory.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/EmployeePayHistory/2024-02-21T11:00:19.5488464/HumanResources_EmployeePayHistory.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/JobCandidate/2024-02-21T11:00:35.0597455/HumanResources_JobCandidate.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Shift/2024-02-21T11:00:52.2700202/HumanResources_Shift.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/Address/2024-02-21T11:01:07.2385191/Person_Address.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/AddressType/2024-02-21T11:01:37.4855778/Person_AddressType.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/BusinessEntity/2024-02-21

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
from datetime import datetime 
import datetime,pytz
from pyspark.sql.functions import substring, col, lit, concat, expr
for file in file_paths:

    df = spark.read.csv(file, header=True, inferSchema=True)
    print(df)
    for column in df.columns:
        
        def encode_to_utf8(text):     
         return text.encode('utf-8') 
         df=df.withColumn(column,encode_to_utf8(column))
        if df.schema[column].dataType == StringType():
            df = df.withColumn(column,upper(df[column]))
        if df.schema[column].dataType == DecimalType() or df.schema[column].dataType == DoubleType():
            df.withColumn(column, format_number(df[column],2))
        if column == 'AverageRate' or column == 'EndOfDayRate':
            df.withColumn(column, floor(df[column]*75))
        fp= 'dbfs:/mnt/bronze-srini/BRONZE/Production/Product/2024-02-21T11:03:01.3756098/Production_Product.csv'
        if file ==fp:
            df = spark.read.csv(file, header=True, inferSchema=True)  
        if column == 'ProductNumber':
            df = df.withColumn('ProductNumber', concat(col('Name'), substring(col('ProductNumber'), 3, 100)))

        f_p= 'dbfs:/mnt/bronze-srini/BRONZE/Person/CountryRegion/2024-02-21T11:03:01.3756098/Person_CountryRegion.csv'  
        if file ==f_p:
            df = spark.read.csv(file, header=True, inferSchema=True)
        if column == 'Name':   
            df=df.withColumn('New_Region_Name', upper(substring('Name', 1,3)))
               # df.show()
    
    current_timestamp = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%dT%H-%M-%S.%f')[:-3]
 
      # Split the file_path string based on '/'
    path_parts = file.split('/')
 
     # Replace 'Bronze' with 'Silver' in the path_parts list
    path_parts[path_parts.index('BRONZE')] = 'Silver'
 
     # Replace the timestamp part with the current timestamp
    path_parts[-2] = current_timestamp
 
     # Join the path_parts list back into a string using '/'
    dynamic_path = '/'.join(path_parts)
    df.write.csv(dynamic_path,header=True, mode="overwrite")

DataFrame[DepartmentID: int, Name: string, GroupName: string, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, NationalIDNumber: int, LoginID: string, OrganizationNode: string, OrganizationLevel: int, JobTitle: string, BirthDate: timestamp, MaritalStatus: string, Gender: string, HireDate: timestamp, SalariedFlag: boolean, VacationHours: int, SickLeaveHours: int, CurrentFlag: boolean, rowguid: string, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, DepartmentID: int, ShiftID: int, StartDate: timestamp, EndDate: timestamp, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, RateChangeDate: timestamp, Rate: double, PayFrequency: int, ModifiedDate: timestamp]
DataFrame[JobCandidateID: string, BusinessEntityID: string, Resume: string, ModifiedDate: string]
DataFrame[ShiftID: int, Name: string, StartTime: timestamp, EndTime: timestamp, ModifiedDate: timestamp]
DataFrame[AddressID: int, AddressLine1: string, AddressLine2: string, City: string, StateProvinceID: int, Po

In [0]:
#f_p= 'dbfs:/mnt/bronze-srini/BRONZE/Person/CountryRegion/2024-02-21T11:03:01.3756098/Person_CountryRegion.csv'  
        #if file ==f_p:
           # df = spark.read.csv(file, header=True, inferSchema=True)
           # if column == 'Name':   
                #df=df.withColumn('New_Region_Name', upper(substring('Name', 1,3)))
               # df.show()

In [0]:
import datetime,pytz
 
file_path = 'dbfs:/mnt/bronze-srini/BRONZE/Person/CountryRegion/2024-02-21T11:03:01.3756098/Person_CountryRegion.csv'
  
# Get the current timestamp
current_timestamp = str(datetime.datetime.now(pytz.timezone('Asia/Kolkata')))#strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
 
# Split the file_path string based on '/'
path_parts = file.split('/')
 
# Replace 'Bronze' with 'Silver' in the path_parts list
path_parts[path_parts.index('BRONZE')] = 'Silver'
 
# Replace the timestamp part with the current timestamp
path_parts[-2] = current_timestamp
 
# Join the path_parts list back into a string using '/'
dynamic_path = '/'.join(path_parts)
 
print(dynamic_path)


dbfs:/mnt/bronze-srini/Silver/Person/CountryRegion/2024-02-29 13:43:36.694178+05:30/Person_CountryRegion.csv


In [0]:
current_timestamp = str(datetime.datetime.now(pytz.timezone('Asia/Kolkata'))) #strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
print(current_timestamp)

2024-02-29 11:34:12.485183+05:30


In [0]:
import datetime as dt



dt_UTC_aware = dt.datetime.now(dt.timezone.utc)
print(dt_UTC_aware)

2024-02-29 09:12:59.237637+00:00


In [0]:
timestamp = datetime.now(pytz.timezone('Asia/Kolkata'))
 

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-1221095670704161>:1
----> 1 timestamp = datetime.now(pytz.timezone('Asia/Kolkata'))

AttributeError: module 'datetime' has no attribute 'now'

In [0]:

def encode_to_utf8(text):     
    return text.encode('utf-8') 
    udf = udf(encode_to_utf8, StringType()) # Apply the UDF to the DataFrame 
columndf_encoded = df.withColumn("encoded_text", encode_udf(df["text"]))
 df=df.withColumn('encode_to_utf8', StringType())



In [0]:
df=df.write.csv(dynamic_path,header=True,mode="overwrite")

In [0]:
df_upper = df.withColumn("text_field_upper", upper(col("text_field")))

In [0]:
# Format numeric_field to a consistent number of decimal places
df_formatted = df.withColumn("formatted_numeric_field", format(col("numeric_field"),2))

# Show the resulting DataFrame
df_formatted.show()

In [0]:
# Define a mapping dictionary for abbreviation expansion
abbreviations_map = {
    "Dr.": "Doctor",
    "Mr.": "Mister",
    "Mrs.": "Missus",
    # Add more abbreviations and their expanded forms as needed
}

# Create a function to expand abbreviations
def expand_abbreviations(text):
    for abbreviation, expansion in abbreviations_map.items():
        text = text.replace(abbreviation, expansion)
    return text

# Register the UDF
expand_abbreviations_udf = spark.udf.register("expand_abbreviations", expand_abbreviations)

# Apply the UDF to expand abbreviations
df_expanded = df.withColumn("expanded_text", expand_abbreviations_udf(col("text_with_abbreviations")))

# Alternatively, you can use PySpark's 'when' function for expansion
for abbreviation, expansion in abbreviations_map.items():
    df_expanded = df_expanded.withColumn("expanded_text", when(col("expanded_text").contains(abbreviation), col("expanded_text").replace(abbreviation, expansion)))

# Show the resulting DataFrame
df_expanded.show()


In [0]:
exchange_rate_pisa_to_inr = 0.4735  # 1 PISA = 0.4735 INR

# Convert PISA to INR
df_converted = df.withColumn("inr_amount", col("pisa_amount") * lit(exchange_rate_pisa_to_inr))


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col



# Sample exchange rate data (you should replace this with actual data)
exchange_rate_data = [
    ("USD", "INR", 75.0),
    # Add more exchange rates as needed
]

# Create DataFrame for exchange rates
exchange_rate_df = spark.createDataFrame(exchange_rate_data, ["USD", "INR", "75.0"])

# Sample data with USD amounts
usd_data = [
    (100,),
    (200,),
    # Add more data as needed
]

# Create DataFrame for USD amounts
usd_df = spark.createDataFrame(usd_data, ["usd_amount"])

# Join USD DataFrame with exchange rate DataFrame
joined_df = usd_df.crossJoin(exchange_rate_df)

# Perform the conversion
converted_df = joined_df.withColumn("inr_amount", col("usd_amount") * col("rate"))

# Optionally round the converted amount
converted_df = converted_df.withColumn("inr_amount_rounded", round(col("inr_amount"), 2))

# Select only relevant columns
result_df = converted_df.select("usd_amount", "inr_amount_rounded")

# Show the result
result_df.show()

# Stop the SparkSession
spark.stop()
